## Star Tracker ML Pipeline ##

In [2]:
import tensorflow as tf
from tensorflow.keras import layers
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import os

# Make numpy values easier to read.
np.set_printoptions(precision=3, suppress=True)

print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.16.2


### Filepaths and Hyperparameters ###

In [3]:
# Data filepath for training
file = "../images_data/mag5_1608_47deg_gray/bins/mag5_1608_47deg_bin_features.csv"          # <-- Configure together
file_descript = "mag5_1608_47deg"     # Magnitude of stars, num classes, camera fov         # <-- Configure together

# Configuration values
validation_split = 0.20
num_epochs = 100
batch_size = 40
saved_model_dir = 'models/'

In [4]:

# TODO: read the header instead of hardcoding it
dataframe = pd.read_csv(file, header=0)
input_size = len(dataframe.columns) - 1
dataset = dataframe.values
X = dataset[:,1:(input_size+1)].astype(int)
Y = dataset[:,0].astype(int)
print("Input Size:", len(X[0]))
num_classes = len(np.unique(Y))
print("Output Size:", num_classes)

Input Size: 10
Output Size: 1608


In [5]:
# Encode class values as integers and then create dataframe with one hot encoding
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)

dummy_Y = tf.one_hot(encoded_Y, depth=num_classes)
print(dummy_Y)



# Unused code from when I was sanity checking
"""
for i in range(100):
    if not (encoder.classes_[dummy_Y.numpy()[i].argmax()] == Y[i]):
        print("Error")
        # print(dummy_Y.numpy()[i].argmax())
        # print(Y[i])
        # print(encoder.classes_[dummy_Y.numpy()[i].argmax()])
"""

"""
# star_labels = np.zeros(num_classes).astype(int)
# get labels in num_classes sized list

# one_hot_array = dummy_Y.numpy()
# # print(Y[0])
# for i in range(len(one_hot_array)):
#     if star_labels[one_hot_array[i].argmax()] == 0:
#         star_labels[one_hot_array[i].argmax()] = Y[i]
# np.set_printoptions(threshold=np.inf)
# print(star_labels)
"""


2024-07-07 13:25:53.284134: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-07 13:25:53.404220: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-07 13:25:53.404281: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-07 13:25:53.406422: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-07 13:25:53.406478: I external/local_xla/xla/stream_executor

tf.Tensor(
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]], shape=(484008, 1608), dtype=float32)


'\n# star_labels = np.zeros(num_classes).astype(int)\n# get labels in num_classes sized list\n\n# one_hot_array = dummy_Y.numpy()\n# # print(Y[0])\n# for i in range(len(one_hot_array)):\n#     if star_labels[one_hot_array[i].argmax()] == 0:\n#         star_labels[one_hot_array[i].argmax()] = Y[i]\n# np.set_printoptions(threshold=np.inf)\n# print(star_labels)\n'

In [6]:
# Create the ML Model
star_model = tf.keras.Sequential([
  layers.Dense(input_size, activation="relu", name="layer1"),
  layers.Dense(64, activation="relu", name="layer2"),
  layers.Dense(128, activation="relu", name="layer3"),
  layers.Dense(1608, activation="softmax", name ="layer4")
])

star_model.compile(loss = "categorical_crossentropy",
                      optimizer = 'adam', metrics = ['accuracy'])

In [7]:
# Model Training
star_model.fit(X, dummy_Y, epochs=num_epochs)

Epoch 1/100


2024-07-07 13:26:32.753606: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 38720640 exceeds 10% of free system memory.
2024-07-07 13:26:32.787700: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 38720640 exceeds 10% of free system memory.
I0000 00:00:1720383993.633384   17791 service.cc:145] XLA service 0x7fa13c00a190 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1720383993.633693   17791 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce RTX 3070 Ti, Compute Capability 8.6
2024-07-07 13:26:33.685348: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-07-07 13:26:33.856796: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8907
I0000 00:00:1720383994.719908   17967 asm_compiler.cc:369] ptxas warning : Registers are spilled to l

   90/15126 ━━━━━━━━━━━━━━━━━━━━ 25s 2ms/step - accuracy: 0.0030 - loss: 7.6325      

I0000 00:00:1720383997.487069   17791 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


15121/15126 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7456 - loss: 1.2540

I0000 00:00:1720384028.894340   18197 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_376', 60 bytes spill stores, 60 bytes spill loads



15126/15126 ━━━━━━━━━━━━━━━━━━━━ 38s 2ms/step - accuracy: 0.7457 - loss: 1.2537
Epoch 2/100
15126/15126 ━━━━━━━━━━━━━━━━━━━━ 29s 2ms/step - accuracy: 0.9635 - loss: 0.1269
Epoch 3/100
15126/15126 ━━━━━━━━━━━━━━━━━━━━ 31s 2ms/step - accuracy: 0.9720 - loss: 0.0911
Epoch 4/100
15126/15126 ━━━━━━━━━━━━━━━━━━━━ 32s 2ms/step - accuracy: 0.9754 - loss: 0.0765
Epoch 5/100
15126/15126 ━━━━━━━━━━━━━━━━━━━━ 31s 2ms/step - accuracy: 0.9779 - loss: 0.0677
Epoch 6/100
15126/15126 ━━━━━━━━━━━━━━━━━━━━ 31s 2ms/step - accuracy: 0.9783 - loss: 0.0643
Epoch 7/100
15126/15126 ━━━━━━━━━━━━━━━━━━━━ 32s 2ms/step - accuracy: 0.9795 - loss: 0.0596
Epoch 8/100
15126/15126 ━━━━━━━━━━━━━━━━━━━━ 31s 2ms/step - accuracy: 0.9802 - loss: 0.0568
Epoch 9/100
15126/15126 ━━━━━━━━━━━━━━━━━━━━ 32s 2ms/step - accuracy: 0.9806 - loss: 0.0556
Epoch 10/100
15126/15126 ━━━━━━━━━━━━━━━━━━━━ 32s 2ms/step - accuracy: 0.9806 - loss: 0.0552
Epoch 11/100
15126/15126 ━━━━━━━━━━━━━━━━━━━━ 32s 2ms/step - accuracy: 0.9810 - loss: 0.054

In [8]:
# Save Model
star_model.summary()

star_model.save(saved_model_dir + file_descript +'.keras')
if not os.path.exists(saved_model_dir + file_descript +'_saved_model/'):
    os.makedirs(saved_model_dir + file_descript +'_saved_model/')
star_model.export(saved_model_dir + file_descript +'_saved_model/')

# TODO: tflite conversion is not working.
# # Convert the model
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir + file_descript +'_saved_model/') # path to the SavedModel directory
tflite_model = converter.convert()

# # Save the model.
with open(saved_model_dir + file_descript + '_small.tflite', 'wb') as f:
  f.write(tflite_model)

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ layer1 (Dense)                  │ (None, 10)             │           110 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer2 (Dense)                  │ (None, 64)             │           704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer3 (Dense)                  │ (None, 128)            │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer4 (Dense)                  │ (None, 1608)           │       207,432 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 649,700 (2.48 MB)

 Trainable params: 216,566 (845.96 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 433,134 (1.65 MB)

INFO:tensorflow:Assets written to: models/mag5_1608_47deg_saved_model/assets


INFO:tensorflow:Assets written to: models/mag5_1608_47deg_saved_model/assets


Saved artifact at 'models/mag5_1608_47deg_saved_model/'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 10), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 1608), dtype=tf.float32, name=None)
Captures:
  140334234585920: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140334234594192: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140334234597184: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140334234800144: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140334234807008: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140334234808416: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140334234807536: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140334235030928: TensorSpec(shape=(), dtype=tf.resource, name=None)


W0000 00:00:1720387154.343746   17409 tf_tfl_flatbuffer_helpers.cc:390] Ignored output_format.
W0000 00:00:1720387154.343779   17409 tf_tfl_flatbuffer_helpers.cc:393] Ignored drop_control_dependency.
2024-07-07 14:19:14.344679: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: models/mag5_1608_47deg_saved_model/
2024-07-07 14:19:14.345130: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-07-07 14:19:14.345138: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: models/mag5_1608_47deg_saved_model/
2024-07-07 14:19:14.353097: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is not enabled
2024-07-07 14:19:14.353911: I tensorflow/cc/saved_model/loader.cc:234] Restoring SavedModel bundle.
2024-07-07 14:19:14.378204: I tensorflow/cc/saved_model/loader.cc:218] Running initialization op on SavedModel bundle at path: models/mag5_1608_47deg_saved_model/
2024-07-